In [2]:
# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import missingno as msno
import pickle

In [3]:
# Pandas Ayarları
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 170)

In [4]:
# Kendi Oluşturduğum Kütüphaneler
from dsmlbc_eda import DsmlbcEDA # EDA modülü
from data_prep import * # Veri ön işleme fonksiyonlarının oluşturulduğu modül

In [5]:
def load_data():
    """
    Diabetes veri setini döndürür.
    """
    return pd.read_csv("diabetes.csv")

In [ ]:
df = load_data()